In [ ]:
!pip install -q wfdb pywavelets torchinfo

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.8/163.8 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 144.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.3.3 which is incompatible.


In [ ]:
import datetime
import wfdb
from wfdb import get_record_list, dl_database
import pywt
import seaborn
import matplotlib.pyplot as plt
import csv
import pandas as pd
import numpy as np
from scipy import signal
import torch
import numpy as np
from tqdm import tqdm
from sklearn.metrics import fbeta_score, roc_auc_score, roc_curve, roc_curve, auc
from torch.utils.data import TensorDataset, DataLoader

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import zipfile
import re

# Define the path to the zip file in Google Drive
zip_file_path = '/content/drive/MyDrive/CINC2020.zip'

# Define the extraction path
extract_path = '/content/CINC'

# Create the extraction directory if it doesn't exist
os.makedirs(extract_path, exist_ok=True)

# Check if the zip file exists
if os.path.exists(zip_file_path):
    # Extract the zip file
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)
    print(f"Successfully extracted {zip_file_path} to {extract_path}")
else:
    print(f"Error: {zip_file_path} not found.")

Successfully extracted /content/drive/MyDrive/CINC2020.zip to /content/CINC


In [ ]:
from concurrent.futures import ProcessPoolExecutor


def denoise_ecg(signal, wavelet='db5', level=9):
    """
    Denoise ECG signal (multi-lead) using wavelet transform.
    signal: shape (n_leads, n_samples)
    """
    coeffs = pywt.wavedec(signal, wavelet, level=level, axis=1)
    # Universal threshold
    threshold = (np.median(np.abs(coeffs[-1])) / 0.6745) * (np.sqrt(2 * np.log(signal.shape[1])))

    coeffs[-1].fill(0)
    coeffs[-2].fill(0)

    for i in range(1, len(coeffs) - 2):
        coeffs[i] = pywt.threshold(coeffs[i], threshold)

    rdata = pywt.waverec(coeffs, wavelet)

    return np.nan_to_num(rdata, nan=0.0, posinf=0.0, neginf=0.0)


def pad_or_truncate(arr, target_len):
    """
    Pad or truncate signal to fixed length.
    arr: shape (n_leads, n_samples)
    """
    if arr.shape[1] < target_len:
        pad_width = target_len - arr.shape[1]
        return np.pad(arr, ((0, 0), (0, pad_width)), mode='constant')
    return arr[:, :target_len]


def process_record(file_name, folder, available_labels):
    final_data_path = f"CINC/classification-of-12-lead-ecgs-the-physionetcomputing-in-cardiology-challenge-2020-1.0.2/{folder}/{file_name}"
    hdr = wfdb.rdheader(final_data_path)

    for comment in hdr.comments:
        if comment.startswith('Dx:') and "," not in comment:
            label = comment.replace('Dx: ', '')
            if label not in available_labels:
                return None

            record = wfdb.rdrecord(final_data_path)
            signal = np.nan_to_num(record.p_signal, nan=0.0)

            signal = signal.T  # shape (n_leads, n_samples)

            denoised = denoise_ecg(signal)  # vectorized
            return denoised, label
    return None

def load_and_preprocess_data():
    print("Loading and preprocessing ECG data...")

    RECORDS = "CINC/classification-of-12-lead-ecgs-the-physionetcomputing-in-cardiology-challenge-2020-1.0.2/RECORDS"
    TARGET_LENGTH = 5000
    available_labels = [
        '10370003', '164889003', '164909002', '164934002',
        '270492004', '284470004', '426177001', '426783006',
        '427084000', '427393009', '59118001'
    ]

    X, Y = [], []

    with open(RECORDS) as f:
        folders_list = f.read()
        FOLDERS = folders_list.strip().splitlines()

    tasks = []
    with ProcessPoolExecutor() as ex:
        for folder in FOLDERS:
            if folder in ["training/ptb/g1/", "training/st_petersburg_incart/g1/"]:
                continue

            with open(f"CINC/classification-of-12-lead-ecgs-the-physionetcomputing-in-cardiology-challenge-2020-1.0.2/{folder}/RECORDS") as r:
                files = r.read().strip().splitlines()[:-1]

            for file_name in files:
                tasks.append(ex.submit(process_record, file_name, folder, available_labels))

        results = [t.result() for t in tasks if t.result() is not None]

    for signal, label in results:
        truncated = pad_or_truncate(signal, TARGET_LENGTH)
        X.append(truncated)
        Y.append(label)

    print("Data loading finished")

    # Convert to arrays
    X = np.array(X)
    Y = np.array(Y)

    label2idx = {label: i for i, label in enumerate(available_labels)}
    Y_idx = np.array([label2idx[y] for y in Y])
    Y_one_hot = np.eye(len(available_labels))[Y_idx]

    return X, Y_one_hot

In [ ]:
standardized_arrays, Y_one_hot_final = load_and_preprocess_data()

In [ ]:
standardized_arrays = standardized_arrays.transpose(0, 2, 1)

In [ ]:
standardized_arrays.shape, Y_one_hot_final.shape

((14530, 5000, 12), (14530, 11))

In [ ]:
X = torch.tensor(standardized_arrays, dtype=torch.float32)   # (14530, 5000, 1)
y = torch.tensor(np.array(Y_one_hot_final).argmax(axis=1), dtype=torch.long)  # (14530,)

In [ ]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F

# -------------------------
# Channel Attention (CBAM)
# -------------------------
class ChannelAttention(nn.Module):
    def __init__(self, channels, ratio=8):
        super().__init__()
        hidden = max(1, channels // ratio)
        # Shared MLP
        self.mlp = nn.Sequential(
            nn.Linear(channels, hidden, bias=True),
            nn.ReLU(inplace=True),
            nn.Linear(hidden, channels, bias=True)
        )

    def forward(self, x):
        # x: (B, C, L)
        b, c, l = x.shape

        # Global average pooling and max pooling across temporal dimension -> (B, C)
        avg_pool = F.adaptive_avg_pool1d(x, 1).view(b, c)   # (B, C)
        max_pool = F.adaptive_max_pool1d(x, 1).view(b, c)   # (B, C)

        # Shared MLP applied to both
        avg_out = self.mlp(avg_pool)  # (B, C)
        max_out = self.mlp(max_pool)  # (B, C)

        # Combine and sigmoid
        out = torch.sigmoid(avg_out + max_out).unsqueeze(-1)  # (B, C, 1)

        # Scale input
        return x * out  # broadcasting over length

# -------------------------
# Spatial Attention (CBAM)
# -------------------------
class SpatialAttention(nn.Module):
    def __init__(self, kernel_size=7):
        super().__init__()
        padding = (kernel_size - 1) // 2
        self.conv = nn.Conv1d(in_channels=2, out_channels=1,
                              kernel_size=kernel_size, padding=padding, bias=False)

    def forward(self, x):
        # x: (B, C, L)
        # channel-wise avg and max -> (B, 1, L)
        avg_pool = torch.mean(x, dim=1, keepdim=True)
        max_pool, _ = torch.max(x, dim=1, keepdim=True)

        cat = torch.cat([avg_pool, max_pool], dim=1)  # (B, 2, L)
        attn = torch.sigmoid(self.conv(cat))  # (B, 1, L)
        return x * attn  # broadcast over channels

# -------------------------
# Transformer Encoder Block
# -------------------------
class TransformerEncoderBlock(nn.Module):
    def __init__(self, d_model, num_heads, dff, dropout_rate=0.1):
        super().__init__()
        # Use batch_first=True so inputs are (B, L, D)
        self.mha = nn.MultiheadAttention(embed_dim=d_model, num_heads=num_heads,
                                         dropout=dropout_rate, batch_first=True)
        self.dropout1 = nn.Dropout(dropout_rate)
        self.norm1 = nn.LayerNorm(d_model, eps=1e-6)

        self.ffn = nn.Sequential(
            nn.Linear(d_model, dff),
            nn.ReLU(inplace=True),
            nn.Linear(dff, d_model)
        )
        self.dropout2 = nn.Dropout(dropout_rate)
        self.norm2 = nn.LayerNorm(d_model, eps=1e-6)

    def forward(self, x, src_mask=None, src_key_padding_mask=None):
        # x: (B, L, D)
        attn_out, _ = self.mha(x, x, x,
                              attn_mask=src_mask,
                              key_padding_mask=src_key_padding_mask)
        attn_out = self.dropout1(attn_out)
        x = self.norm1(x + attn_out)

        ffn_out = self.ffn(x)
        ffn_out = self.dropout2(ffn_out)
        x = self.norm2(x + ffn_out)
        return x  # (B, L, D)

# -------------------------
# Positional Encoding (sinusoidal)
# -------------------------
def sinusoidal_positional_encoding(seq_len, d_model, device=None, dtype=None):
    if device is None:
        device = torch.device('cpu')
    pe = torch.zeros(seq_len, d_model, device=device, dtype=dtype)
    position = torch.arange(0, seq_len, dtype=torch.float, device=device).unsqueeze(1)
    div_term = torch.exp(torch.arange(0, d_model, 2, device=device, dtype=torch.float)
                         * -(math.log(10000.0) / d_model))
    pe[:, 0::2] = torch.sin(position * div_term)
    pe[:, 1::2] = torch.cos(position * div_term)
    return pe.unsqueeze(0)  # (1, seq_len, d_model)

# -------------------------
# Full Model
# -------------------------
class ECGModel(nn.Module):
    def __init__(self, sequence_length=5000, num_channels=12,
                 d_model=128, num_heads=4, dff=128, dropout_rate=0.2,
                 num_classes=11, apply_softmax=False):
        super().__init__()
        self.apply_softmax = apply_softmax

        self.conv1 = nn.Conv1d(in_channels=num_channels, out_channels=32, kernel_size=21, padding=(21-1)//2)
        self.bn1 = nn.BatchNorm1d(32)
        self.ca1 = ChannelAttention(32, ratio=8)
        self.pool1 = nn.MaxPool1d(kernel_size=3, stride=2, padding=1) 

        self.conv2 = nn.Conv1d(32, 32, kernel_size=23, padding=(23-1)//2)
        self.bn2 = nn.BatchNorm1d(32)
        self.ca2 = ChannelAttention(32, ratio=8)
        self.pool2 = nn.MaxPool1d(kernel_size=3, stride=2, padding=1)

        self.conv3 = nn.Conv1d(32, 64, kernel_size=25, padding=(25-1)//2)
        self.bn3 = nn.BatchNorm1d(64)
        self.ca3 = ChannelAttention(64, ratio=8)
        self.pool3 = nn.MaxPool1d(kernel_size=3, stride=2, padding=1)

        self.conv4 = nn.Conv1d(64, 128, kernel_size=27, padding=(27-1)//2)
        self.bn4 = nn.BatchNorm1d(128)
        self.ca4 = ChannelAttention(128, ratio=8)

        if 128 != d_model:
            self.project_to_d_model = nn.Conv1d(128, d_model, kernel_size=1)
        else:
            self.project_to_d_model = None

        # Transformer encoder
        self.transformer_block = TransformerEncoderBlock(d_model=d_model, num_heads=num_heads,
                                                         dff=dff, dropout_rate=dropout_rate)

        self.flatten = nn.Flatten()
        self.fc1 = None  
        self.dropout = nn.Dropout(dropout_rate)
        self.fc_out = nn.Linear(128, num_classes)

        self._init_weights()

        self._configured = False
        self.d_model = d_model

    def _init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv1d):
                nn.init.kaiming_normal_(m.weight, nonlinearity='relu')
                if m.bias is not None:
                    nn.init.zeros_(m.bias)
            elif isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.zeros_(m.bias)
            elif isinstance(m, nn.BatchNorm1d):
                nn.init.ones_(m.weight)
                nn.init.zeros_(m.bias)

    def _init_fc_head(self, sample_batch, device):
        self.eval()
        with torch.no_grad():
            x = sample_batch.to(device)
            features = self._forward_features(x) 
            flat = features.reshape(features.size(0), -1)
            in_features = flat.size(1)
            self.fc1 = nn.Linear(in_features, 128).to(device)
            nn.init.xavier_uniform_(self.fc1.weight)
            nn.init.zeros_(self.fc1.bias)
            self._configured = True


    def _forward_features(self, x):
        x = x.permute(0, 2, 1)  # -> (B, C, L)

        x = self.conv1(x)
        x = self.bn1(x)
        x = F.relu(x, inplace=True)
        x = self.ca1(x)
        x = self.pool1(x)

        x = self.conv2(x)
        x = self.bn2(x)
        x = F.relu(x, inplace=True)
        x = self.ca2(x)
        x = self.pool2(x)

        x = self.conv3(x)
        x = self.bn3(x)
        x = F.relu(x, inplace=True)
        x = self.ca3(x)
        x = self.pool3(x)

        x = self.conv4(x)
        x = self.bn4(x)
        x = F.relu(x, inplace=True)
        x = self.ca4(x)
        # x: (B, channels, L_final)

        if self.project_to_d_model is not None:
            x = self.project_to_d_model(x)  # (B, d_model, L_final)

        # Transpose for transformer: (B, L, D)
        x = x.permute(0, 2, 1)

        seq_len = x.size(1)
        pe = sinusoidal_positional_encoding(seq_len, self.d_model, device=x.device, dtype=x.dtype)
        x = x + pe  # (B, L, D)

        # Transformer encoder block
        x = self.transformer_block(x)  # (B, L, D)
        return x

    def forward(self, x):
        device = x.device

        if not self._configured:
            self._init_fc_head(x, device)

        features = self._forward_features(x)  # (B, L, D)
        flat = features.reshape(features.size(0), -1)  # flatten
        x = F.relu(self.fc1(flat))
        x = self.dropout(x)
        logits = self.fc_out(x)
        if self.apply_softmax:
            return F.softmax(logits, dim=-1)
        return logits


sequence_length = 5000
num_channels = 12
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = ECGModel(sequence_length=sequence_length, num_channels=num_channels,
                  d_model=128, num_heads=4, dff=128, dropout_rate=0.2,
                  num_classes=11, apply_softmax=False).to(device)

# dummy batch with shape (batch, seq_len, channels)
dummy = torch.randn(2, sequence_length, num_channels, device=device)
out = model(dummy)  # out shape: (2, 11) logits
print("output shape:", out.shape)

output shape: torch.Size([2, 11])


In [ ]:
from torchinfo import summary

print(X.shape)

dummy_input = torch.randn(1, X.shape[1], X.shape[2]).to(device)

summary(model, input_data=dummy_input, device=str(device))


torch.Size([14530, 5000, 12])


Layer (type:depth-idx)                   Output Shape              Param #
ECGModel                                 [1, 11]                   --
├─Conv1d: 1-1                            [1, 32, 5000]             8,096
├─BatchNorm1d: 1-2                       [1, 32, 5000]             64
├─ChannelAttention: 1-3                  [1, 32, 5000]             --
│    └─Sequential: 2-1                   [1, 32]                   --
│    │    └─Linear: 3-1                  [1, 4]                    132
│    │    └─ReLU: 3-2                    [1, 4]                    --
│    │    └─Linear: 3-3                  [1, 32]                   160
│    └─Sequential: 2-2                   [1, 32]                   (recursive)
│    │    └─Linear: 3-4                  [1, 4]                    (recursive)
│    │    └─ReLU: 3-5                    [1, 4]                    --
│    │    └─Linear: 3-6                  [1, 32]                   (recursive)
├─MaxPool1d: 1-4                         [1, 32, 2500

In [ ]:
from sklearn.metrics import roc_auc_score, precision_score, recall_score, f1_score, accuracy_score
import random

def evaluate_on_loader(model, loader, criterion, device, num_classes):
    model.eval()
    running_loss = 0.0
    all_labels, all_preds, all_probs = [], [], []

    with torch.no_grad():
        for xb, yb in loader:
            xb = xb.to(device)
            yb = yb.to(device)

            logits = model(xb)  # (B, C)
            if torch.isnan(logits).any() or torch.isinf(logits).any():
               print(f"Found NaN/Inf in logits at batch {i}")
               any_nan = True
            loss = criterion(logits, yb)
            running_loss += loss.item() * xb.size(0)

            probs = torch.softmax(logits, dim=1).cpu().numpy()
            preds = logits.argmax(dim=1).cpu().numpy()
            labels = yb.cpu().numpy()

            all_labels.extend(labels)
            all_preds.extend(preds)
            all_probs.extend(probs)

    avg_loss = running_loss / len(all_labels)
    acc = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, average="weighted", zero_division=0)
    recall = recall_score(all_labels, all_preds, average="weighted", zero_division=0)
    f1 = f1_score(all_labels, all_preds, average="weighted", zero_division=0)

    try:
        y_true = np.eye(num_classes)[all_labels]
        y_score = np.array(all_probs)

        auc_list = []
        for i in range(num_classes):
            if np.any(y_true[:, i]):  # class i exists
                auc_list.append(roc_auc_score(y_true[:, i], y_score[:, i]))
        if auc_list:
            auc = np.mean(auc_list)
        else:
            auc = float("nan")
    except ValueError:
        auc = float("nan")

    return avg_loss, acc, precision, recall, auc, f1


def confusion_matrix(preds, targets, num_classes):
    cm = np.zeros((num_classes, num_classes), dtype=int)
    for p, t in zip(preds, targets):
        cm[int(t), int(p)] += 1
    return cm

def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)

In [ ]:
from torch import optim
import time
from sklearn.model_selection import train_test_split

batch_size = 16
epochs = 20
lr = 1e-3

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_classes = int(torch.max(y).item() + 1)
all_results = []


seeds = list(range(10))

for seed_idx, seed in enumerate(seeds, 1):
    print(f"\n===== SEED {seed_idx}: {seed} =====")
    set_seed(seed)

    N = X.shape[0]
    assert N == y.shape[0], "X and y must have same first dimension"
    num_classes = int(torch.max(y).item() + 1)
    print(f"N={N}, num_classes={num_classes}, device={device}")

    all_indices = list(range(len(X)))
    all_labels = y.numpy()

    trainval_indices, test_indices = train_test_split(
        all_indices,
        test_size=0.1,
        random_state=seed,
        stratify=all_labels
    )

    trainval_labels = [all_labels[i] for i in trainval_indices]

    train_indices, val_indices = train_test_split(
        trainval_indices,
        test_size=0.1,
        random_state=seed,
        stratify=trainval_labels
    )

    # Create datasets
    train_ds = TensorDataset(X[train_indices], y[train_indices])
    val_ds = TensorDataset(X[val_indices], y[val_indices])
    test_ds = TensorDataset(X[test_indices], y[test_indices])

    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, drop_last=True)
    val_loader = DataLoader(val_ds, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_ds, batch_size=batch_size, shuffle=False)

    # Initialize model
    model = ECGModel(sequence_length=X.shape[1], num_channels=X.shape[2],
                     d_model=128, num_heads=4, dff=128, dropout_rate=0.2,
                     num_classes=num_classes, apply_softmax=False).to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = torch.nn.CrossEntropyLoss()
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2)

    train_acc_list, val_acc_list = [], []

    for epoch in range(1, epochs+1):
        model.train()
        running_correct = 0
        seen = 0

        for xb, yb in train_loader:
            xb, yb = xb.to(device), yb.to(device)
            optimizer.zero_grad()
            logits = model(xb)
            loss = criterion(logits, yb)
            loss.backward()
            optimizer.step()

            running_correct += (logits.argmax(dim=1) == yb).sum().item()
            seen += xb.size(0)

        train_acc = running_correct / seen
        train_acc_list.append(train_acc)

        val_loss, val_acc, val_precision, val_recall, val_auc, val_f1 = evaluate_on_loader(
            model, val_loader, criterion, device, num_classes
        )
        val_acc_list.append(val_acc)
        scheduler.step(val_loss)

        print(f"Epoch {epoch}/{epochs} - Train Acc: {train_acc:.4f}, "
              f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}, Val F1: {val_f1:.4f}")

    # Test evaluation after final epoch using updated function
    test_loss, test_acc, test_precision, test_recall, test_auc, test_f1 = evaluate_on_loader(
        model, test_loader, criterion, device, num_classes
    )

    print(f"Seed {seed} | test_loss: {test_loss:.4f} | test_acc: {test_acc:.4f} | "
          f"AUC: {test_auc:.4f} | Precision: {test_precision:.4f} | "
          f"Recall: {test_recall:.4f} | F1: {test_f1:.4f}")

    all_results.append({
        'seed': seed,
        'train_acc_list': train_acc_list,
        'val_acc_list': val_acc_list,
        'test_acc': test_acc,
        'test_loss': test_loss,
        'test_auc': test_auc,
        'test_precision': test_precision,
        'test_recall': test_recall,
        'test_f1': test_f1
    })

# Summary across seeds
test_accs = [r['test_acc'] for r in all_results]
test_aucs = [r['test_auc'] for r in all_results if not np.isnan(r['test_auc'])]
test_precisions = [r['test_precision'] for r in all_results]
test_recalls = [r['test_recall'] for r in all_results]
test_f1s = [r['test_f1'] for r in all_results]

print(f"\n===== SUMMARY ACROSS {len(seeds)} SEEDS =====")
print(f"Average Test Accuracy: {np.mean(test_accs):.4f} ± {np.std(test_accs):.4f}")
if test_aucs:
    print(f"Average Test AUC: {np.mean(test_aucs):.4f} ± {np.std(test_aucs):.4f}")
else:
    print(f"Average Test AUC: Could not be calculated")
print(f"Average Test Precision: {np.mean(test_precisions):.4f} ± {np.std(test_precisions):.4f}")
print(f"Average Test Recall: {np.mean(test_recalls):.4f} ± {np.std(test_recalls):.4f}")
print(f"Average Test F1: {np.mean(test_f1s):.4f} ± {np.std(test_f1s):.4f}")



===== SEED 1: 0 =====
N=14530, num_classes=11, device=cuda
Epoch 1/20 - Train Acc: 0.7327, Val Loss: 0.8637, Val Acc: 0.7699, Val F1: 0.7072
Epoch 2/20 - Train Acc: 0.7858, Val Loss: 0.6245, Val Acc: 0.8234, Val F1: 0.7927
Epoch 3/20 - Train Acc: 0.8349, Val Loss: 0.5018, Val Acc: 0.8486, Val F1: 0.8320
Epoch 4/20 - Train Acc: 0.8543, Val Loss: 0.4735, Val Acc: 0.8624, Val F1: 0.8475
Epoch 5/20 - Train Acc: 0.8665, Val Loss: 0.4823, Val Acc: 0.8563, Val F1: 0.8366
Epoch 6/20 - Train Acc: 0.8755, Val Loss: 0.5249, Val Acc: 0.8341, Val F1: 0.8365
Epoch 7/20 - Train Acc: 0.8892, Val Loss: 0.4489, Val Acc: 0.8685, Val F1: 0.8610
Epoch 8/20 - Train Acc: 0.8921, Val Loss: 0.4293, Val Acc: 0.8777, Val F1: 0.8696
Epoch 9/20 - Train Acc: 0.8959, Val Loss: 0.4138, Val Acc: 0.8838, Val F1: 0.8801
Epoch 10/20 - Train Acc: 0.9017, Val Loss: 0.4430, Val Acc: 0.8899, Val F1: 0.8792
Epoch 11/20 - Train Acc: 0.9094, Val Loss: 0.4570, Val Acc: 0.8746, Val F1: 0.8669
Epoch 12/20 - Train Acc: 0.9127, Val

In [ ]:
!pip install thop calflops -q

In [ ]:
import calflops

flops, macs, params = calflops.calculate_flops(model=model,
                                      input_shape=(1, 5000, 12),
                                      output_as_string=True,
                                      output_precision=4)
print("Model FLOPs:%s   MACs:%s   Params:%s \n" %(flops, macs, params))


------------------------------------- Calculate Flops Results -------------------------------------
Notations:
number of parameters (Params), number of multiply-accumulate operations(MACs),
number of floating-point operations (FLOPs), floating-point operations per second (FLOPS),
fwd FLOPs (model forward propagation FLOPs), bwd FLOPs (model backward propagation FLOPs),
default model backpropagation takes 2.00 times as much computation as forward propagation.

Total Training Params:                                                  10.65 M 
fwd MACs:                                                               373.133 MMACs
fwd FLOPs:                                                              751.428 MFLOPS
fwd+bwd MACs:                                                           1.1194 GMACs
fwd+bwd FLOPs:                                                          2.2543 GFLOPS

-------------------------------- Detailed Calculated FLOPs Results --------------------------------
Each modu

In [ ]:
import time
from thop import profile, clever_format
from torchinfo import summary


def analyze_model_complexity(model, input_size=(1, 5000, 12), device='cuda'):
    """
    Analyze model complexity: parameters, FLOPs, memory

    Args:
        model: Your HANWithAttention model
        input_size: (batch, segments, timesteps, channels)
        device: 'cuda' or 'cpu'
    """
    print("="*80)
    print("MODEL COMPLEXITY ANALYSIS")
    print("="*80)

    model = model.to(device)
    model.eval()

    # 1. Parameter count
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    non_trainable_params = total_params - trainable_params

    print(f"\nParameter Count:")
    print(f"   Total Parameters:       {total_params:,}")
    print(f"   Trainable Parameters:   {trainable_params:,}")
    print(f"   Non-trainable Parameters: {non_trainable_params:,}")
    print(f"   Model Size (MB):        {total_params * 4 / (1024**2):.2f}")  # 4 bytes per float32

    # 2. FLOPs calculation
    dummy_input = torch.randn(input_size).to(device)

    try:
        macs, params = profile(model, inputs=(dummy_input,), verbose=False)
        macs, params = clever_format([macs, params], "%.3f")
        print(f"\nFLOPs (Floating Point Operations):")
        print(f"   MACs (Multiply-Accumulate): {macs}")
        print(f"   Parameters (thop):          {params}")
    except Exception as e:
        print(f"\nFLOPs calculation failed: {e}")

    # 3. Layer-wise parameter breakdown
    print(f"\nLayer-wise Parameter Breakdown:")
    print(f"{'Layer Name':<40} {'Parameters':>15} {'% of Total':>12}")
    print("-"*70)

    for name, param in model.named_parameters():
        if param.requires_grad:
            param_count = param.numel()
            percentage = 100 * param_count / trainable_params
            print(f"{name:<40} {param_count:>15,} {percentage:>11.2f}%")

    # 4. Detailed model summary
    print(f"\nDetailed Model Architecture:")
    summary(model,
            input_size=input_size,
            col_names=["input_size", "output_size", "num_params", "mult_adds"],
            depth=4,
            device=device,
            verbose=0)

    return {
        'total_params': total_params,
        'trainable_params': trainable_params,
        'model_size_mb': total_params * 4 / (1024**2)
    }

analyze_model_complexity(model)

MODEL COMPLEXITY ANALYSIS

📊 Parameter Count:
   Total Parameters:       10,651,819
   Trainable Parameters:   10,651,819
   Non-trainable Parameters: 0
   Model Size (MB):        40.63

🔢 FLOPs (Floating Point Operations):
   MACs (Multiply-Accumulate): 334.413M
   Parameters (thop):          10.586M

📋 Layer-wise Parameter Breakdown:
Layer Name                                    Parameters   % of Total
----------------------------------------------------------------------
conv1.weight                                       8,064        0.08%
conv1.bias                                            32        0.00%
bn1.weight                                            32        0.00%
bn1.bias                                              32        0.00%
ca1.mlp.0.weight                                     128        0.00%
ca1.mlp.0.bias                                         4        0.00%
ca1.mlp.2.weight                                     128        0.00%
ca1.mlp.2.bias                 

{'total_params': 10651819,
 'trainable_params': 10651819,
 'model_size_mb': 40.63346481323242}

In [ ]:
# From: https://discuss.pytorch.org/t/finding-model-size/130275

param_size = 0
for param in model.parameters():
    param_size += param.nelement() * param.element_size()
buffer_size = 0
for buffer in model.buffers():
    buffer_size += buffer.nelement() * buffer.element_size()

size_all_mb = (param_size + buffer_size) / 1024**2
print('model size: {:.3f}MB'.format(size_all_mb))


model size: 40.636MB


In [ ]:
import time
import numpy as np
import torch

# =============================================================================
# Benchmark Environment Setup
# =============================================================================

def setup_benchmark_env():
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True
    torch.backends.cuda.matmul.allow_tf32 = False
    torch.backends.cudnn.allow_tf32 = False

def time_forward(model, inputs, device='cuda'):
    if device == 'cuda':
        torch.cuda.synchronize()
        start = torch.cuda.Event(enable_timing=True)
        end = torch.cuda.Event(enable_timing=True)

        start.record()
        _ = model(inputs)
        end.record()

        torch.cuda.synchronize()
        return start.elapsed_time(end) / 1000.0  # seconds
    else:
        t0 = time.time()
        _ = model(inputs)
        return time.time() - t0


# =============================================================================
# Batch-1 Latency Benchmark
# =============================================================================

def benchmark_latency(
    model,
    input_shape,
    device='cuda',
    runs=100,
    warmup=20
):
    model = model.to(device).eval()

    with torch.no_grad():
        # Warmup
        for _ in range(warmup):
            x = torch.randn(input_shape, device=device)
            _ = model(x)
            if device == 'cuda':
                torch.cuda.synchronize()

        times = []
        for _ in range(runs):
            x = torch.randn(input_shape, device=device)
            elapsed = time_forward(model, x, device)
            times.append(elapsed)

    times = np.array(times)

    return {
        "mean_ms": times.mean() * 1000,
        "std_ms": times.std() * 1000,
        "median_ms": np.median(times) * 1000,
        "min_ms": times.min() * 1000,
        "max_ms": times.max() * 1000,
    }


# =============================================================================
# Batch-N Throughput Benchmark
# =============================================================================

def benchmark_throughput(
    model,
    input_shape,
    batch_size,
    device='cuda',
    runs=100,
    warmup=20
):
    model = model.to(device).eval()
    shape = (batch_size,) + input_shape[1:]

    with torch.no_grad():
        # Warmup
        for _ in range(warmup):
            x = torch.randn(shape, device=device)
            _ = model(x)
            if device == 'cuda':
                torch.cuda.synchronize()

        times = []
        for _ in range(runs):
            x = torch.randn(shape, device=device)
            elapsed = time_forward(model, x, device)
            times.append(elapsed)

    times = np.array(times)

    return {
        "batch_time_ms": times.mean() * 1000,
        "per_sample_ms": (times.mean() / batch_size) * 1000,
        "samples_per_sec": batch_size / times.mean()
    }

def run_inference_benchmark(
    model,
    input_shape=(1, 5000, 12),
    batch_size=16,
    device='cuda'
):
    setup_benchmark_env()

    print("=" * 80)
    print("INFERENCE BENCHMARK")
    print("=" * 80)
    print(f"Device: {device}")
    print(f"Input shape: {input_shape}")
    print(f"Throughput batch size: {batch_size}")

    latency = benchmark_latency(
        model=model,
        input_shape=input_shape,
        device=device
    )

    throughput = benchmark_throughput(
        model=model,
        input_shape=input_shape,
        batch_size=batch_size,
        device=device
    )

    print("\nBatch-1 Latency (ms):")
    for k, v in latency.items():
        print(f"  {k:>10}: {v:.3f}")

    print("\nBatch-N Throughput:")
    for k, v in throughput.items():
        if "ms" in k:
            print(f"  {k:>16}: {v:.3f}")
        else:
            print(f"  {k:>16}: {v:.2f}")

    return {
        "latency": latency,
        "throughput": throughput
    }

results = run_inference_benchmark(
    model,
    input_shape=(1, 5000, 12),
    batch_size=16,
    device='cpu'
)


/usr/local/lib/python3.12/dist-packages/torch/backends/__init__.py:46: UserWarning: Please use the new API settings to control TF32 behavior, such as torch.backends.cudnn.conv.fp32_precision = 'tf32' or torch.backends.cuda.matmul.fp32_precision = 'ieee'. Old settings, e.g, torch.backends.cuda.matmul.allow_tf32 = True, torch.backends.cudnn.allow_tf32 = True, allowTF32CuDNN() and allowTF32CuBLAS() will be deprecated after Pytorch 2.9. Please see https://pytorch.org/docs/main/notes/cuda.html#tensorfloat-32-tf32-on-ampere-and-later-devices (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:80.)
  self.setter(val)


INFERENCE BENCHMARK
Device: cpu
Input shape: (1, 5000, 12)
Throughput batch size: 16

Batch-1 Latency (ms):
     mean_ms: 7.674
      std_ms: 0.426
   median_ms: 7.567
      min_ms: 7.402
      max_ms: 10.701

Batch-N Throughput:
     batch_time_ms: 78.613
     per_sample_ms: 4.913
   samples_per_sec: 203.53


In [ ]:
results = run_inference_benchmark(
    model,
    input_shape=(1, 5000, 12),
    batch_size=16,
    device='cuda'
)

INFERENCE BENCHMARK
Device: cuda
Input shape: (1, 5000, 12)
Throughput batch size: 16

Batch-1 Latency (ms):
     mean_ms: 2.933
      std_ms: 0.107
   median_ms: 2.903
      min_ms: 2.858
      max_ms: 3.589

Batch-N Throughput:
     batch_time_ms: 5.829
     per_sample_ms: 0.364
   samples_per_sec: 2744.69
